# Topic Modelling using LSA

Algoritma LSA (Latent Semantic Analysis) adalah salah satu algoritma yang dapat digunakan untuk menganalisa hubungan antara sebuah frase/kalimat dengan sekumpulan dokumen.

Pada program ini akan menggunakan data abstrak dari portal tugas akhir trunojoyo program studi Teknik Informatika (https://pta.trunojoyo.ac.id/c_search/byprod/10), berikut code untuk melakukan crawling data:

In [ ]:
# install library beautifulsoup4 untuk melakukan crawling data
pip install beautifulsoup4

In [ ]:
# import library
from bs4 import BeautifulSoup
import requests
import csv

# membuat list, dataAbstract untuk menampung data sementara setelah crawling
# dataFix untuk menampung data yang sudah ditambahkan kolom index dan siap di convert ke csv
dataAbstract = []
dataFix = []

# function crawlAbstract untuk mengambil data judul dan abstract dari halaman detail pta trunojoyo teknik informatika
def crawlAbstract(src):
    # inisialisasi beautifulsoup4     
    global c
    tmp = []
    page = requests.get(src)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # mengambil data judul     
    title = soup.find(class_="title").getText()
    tmp.append(title)
    
    # mengambil data abstract     
    abstractText = soup.p.getText()
    tmp.append(abstractText)
    
    return tmp

# function getLinkToAbstract digunakan untuk mengambil data link menuju halaman detail
# parameter src berisi link halaman daftar tugas akhir
def getLinkToAbstract(src):
    # inisialisasi beautifulsoup4
    global c
    page = requests.get(src)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # mendapatkan semua link menuju halaman detail
    items = soup.find(class_="items").find_all('a')
    # looping setiap link untuk mendapatkan nilai href, 
    # link tersebut digunakan sebagai parameter function crawlAbstract agar mendapat data judul dan abstract
    for item in items:
        if item.get('href') != '#':
            tmp = crawlAbstract(item.get('href'))
            # dataAbstract menampung data sementara hasil crawl
            dataAbstract.append(tmp)


# link halaman pta trunojoyo prodi teknik informatika yang akan di crawl
# halaman ini berisi daftar tugas akhir
link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"
# mengambil data sampai halaman 100
for i in range(1, 101):
    # memindah halaman menuju halaman selanjutnya     
    src = f"https://pta.trunojoyo.ac.id/c_search/byprod/10/{i}"
    # counter untuk melihat progress berapa persen proses crawling
    print(f"Proses-{i}%")
    # memanggil function getLinkToAbstract untuk mendapatkan setiap link ke halaman detail
    getLinkToAbstract(src)

# setelah memperoleh semua data abstract, data tersebut ditampung di list dataAbstract
# data perlu ditambahkan kolom index sebagai id
# looping berikut bertujuan menambahkan kolom index di setiap baris, lalu disimpan di list dataFix
for i in range(1, len(dataAbstract)+1):
    dataAbstract[i-1].insert(0, i)
    dataFix.append(dataAbstract[i-1])

# menyimpan data hasil crawl dengan format csv
header = ['index', 'title','abstract']
with open('dataHasilCrawl.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(header)
    write.writerows(dataFix)
# akan ada file dataHasilCrawl.csv berisi id, judul dan abtrak dari pta trunojoyo teknik informatika sejumlah 500 record
# proses crawling selesai

Tahap selanjutnya melakukan pre-processing data dengan tahapan 1.punctuation removal 2.stemming
1. punctuation removal adalah proses membersihkan teks dari tanda baca dan angka
2. Stemming adalah proses pemetaan dan penguraian bentuk dari suatu kata menjadi bentuk kata dasarnya. Sederhananya, proses mengubah kata berimbuhan menjadi kata dasar, misal kata "membosankan" menjadi "bosan"
3. Stopwords merupakan kata yang diabaikan dalam pemrosesan karena termasuk kata umum yang mempunyai fungsi tapi tidak mempunyai arti.Maksud dari kata umum adalah kata yang frekuensi kemunculannya tinggi, misalnya kata penghubung seperti “dan”, “atau”, “tapi”, “akan” dan lainnya

Install terlebih dahulu library yang akan digunakan:
Sastrawi digunakan untuk proses stemming dan stopword

In [ ]:
pip install sastrawi

In [12]:
import csv # untuk menyimpan hasil dalam format csv
import string 
import re # re : digunakan untuk proses punctuation removal

# memanggil function yang digunakan
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# membuat list untuk menampung data
dataAbstract = []
dataAfterPreprocessing = []

# inisialisasi library sastrawi untuk stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# inisialisasi library sastrawi untuk proses stopword removal
factory2 = StopWordRemoverFactory()
stopword = factory2.create_stop_word_remover()

# untuk counter proses
count = 1

# membaca data dari proses sebelumnya
with open("dataHasilCrawl.csv", "r") as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader:
        if len(row) != 0:
#           data sebelum proses disimpan pada list dataAbstract
            dataAbstract.append(row)

# looping untuk memproses setiap data
for abstract in dataAbstract:
#   ambil data
    tmp = abstract.pop()
#   lakukan case folding (mengubah teks menjadi bentuk standar: huruf kecil)
    tmp = tmp.lower()
#   menghapus angka
    tmp = re.sub(r"\d+", "", tmp)
#   menghapus tanda baca
    tmp = tmp.translate(str.maketrans("","",string.punctuation))
#   menghapus whitespace
    tmp = tmp.strip()
    tmp = re.sub('\s+',' ',tmp)
#   melakukan proses stemming
#     tmp = stemmer.stem(tmp)
#   melakukan proses stopword removal
    tmp = stopword.remove(tmp)
#   menambahkan data ke list dataAfterPreprocessing
    abstract.append(tmp)
    dataAfterPreprocessing.append(abstract)
#   print counter proses
    print(f"Proses:{count}/{len(dataAbstract)}")
    count+=1

# menyimpan data dari list dataAfterPreprocessing ke bentuk csv
header = ['index', 'title','abstract_cleaned']
with open('dataAfterPreprocessing.csv', 'w', encoding="utf-8") as f:
    write = csv.writer(f)
    write.writerow(header)
    write.writerows(dataAfterPreprocessing)
# akan ada file dataAfterPreprocessing.csv berisi id, judul, abtract yang sudah dipreprocessing
# preprocessing sudah selesai

Proses:1/500
Proses:2/500
Proses:3/500
Proses:4/500
Proses:5/500
Proses:6/500
Proses:7/500
Proses:8/500
Proses:9/500
Proses:10/500
Proses:11/500
Proses:12/500
Proses:13/500
Proses:14/500
Proses:15/500
Proses:16/500
Proses:17/500
Proses:18/500
Proses:19/500
Proses:20/500
Proses:21/500
Proses:22/500
Proses:23/500
Proses:24/500
Proses:25/500
Proses:26/500
Proses:27/500
Proses:28/500
Proses:29/500
Proses:30/500
Proses:31/500
Proses:32/500
Proses:33/500
Proses:34/500
Proses:35/500
Proses:36/500
Proses:37/500
Proses:38/500
Proses:39/500
Proses:40/500
Proses:41/500
Proses:42/500
Proses:43/500
Proses:44/500
Proses:45/500
Proses:46/500
Proses:47/500
Proses:48/500
Proses:49/500
Proses:50/500
Proses:51/500
Proses:52/500
Proses:53/500
Proses:54/500
Proses:55/500
Proses:56/500
Proses:57/500
Proses:58/500
Proses:59/500
Proses:60/500
Proses:61/500
Proses:62/500
Proses:63/500
Proses:64/500
Proses:65/500
Proses:66/500
Proses:67/500
Proses:68/500
Proses:69/500
Proses:70/500
Proses:71/500
Proses:72/500
P

Masuk ke tahap penerapan LSA

In [8]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [ ]:
pip install sklearn

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\lenovo-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
stop_words=set(nltk.corpus.stopwords.words('indonesian'))

In [13]:
df=pd.read_csv('./dataAfterPreprocessing.csv')
df.head()

,index,title,abstract_cleaned
0,1,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,sistem informasi akademik siakad merupakan sis...
1,2,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,berjalannya koneksi jaringan komputer lancar g...
2,3,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,web server sebuah perangkat lunak server berfu...
3,4,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,penjadwalan kuliah perguruan tinggi merupakan ...
4,5,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,seiring perkembangan teknologi ada didunia mun...


In [14]:
# drop the publish date.
df.drop(['index'],axis=1,inplace=True)
# drop the publish date.
df.drop(['title'],axis=1,inplace=True)

In [15]:
df.head(10)

,abstract_cleaned
0,sistem informasi akademik siakad merupakan sis...
1,berjalannya koneksi jaringan komputer lancar g...
2,web server sebuah perangkat lunak server berfu...
3,penjadwalan kuliah perguruan tinggi merupakan ...
4,seiring perkembangan teknologi ada didunia mun...
5,gerak pekerja game memiliki genre rts realtime...
6,perkembangan game semakin pesat memberikan ber...
7,sistem pengenalan wajah suatu sistem mengenali...
8,teknologi mobile game beroperating system open...
9,kantor badan kepegawaian kota bangkalan instan...


In [24]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000) # to play with. min_df,max_df,max_features etc...

In [26]:
vect_text=vect.fit_transform(df['abstract_cleaned'].values.astype('U'))

In [27]:
print(vect_text.shape)
print(vect_text)

(500, 1000)
  (0, 354)	0.1184936369291229
  (0, 196)	0.13956580648376873
  (0, 684)	0.1343875919276184
  (0, 179)	0.12133616214140663
  (0, 395)	0.13685884977996102
  (0, 732)	0.09114832069093537
  (0, 867)	0.10565655947056028
  (0, 168)	0.11032302056697307
  (0, 475)	0.09813888102756477
  (0, 923)	0.3081480905267873
  (0, 688)	0.09813888102756477
  (0, 392)	0.1280499695101433
  (0, 366)	0.13211425507553565
  (0, 503)	0.06227680213408083
  (0, 404)	0.12287175495399294
  (0, 929)	0.1343875919276184
  (0, 791)	0.24267232428281327
  (0, 808)	0.13211425507553565
  (0, 329)	0.09684767920074258
  (0, 131)	0.14898423918366568
  (0, 953)	0.11987935314799347
  (0, 784)	0.047360434797127454
  (0, 99)	0.11242780173976039
  (0, 583)	0.08272067498544265
  (0, 365)	0.13685884977996102
  :	:
  (499, 351)	0.06266569569742468
  (499, 39)	0.0923587589179502
  (499, 30)	0.20311354251553534
  (499, 697)	0.05266064595121842
  (499, 117)	0.0938974226708933
  (499, 965)	0.08733535401212822
  (499, 689)	0.128

In [28]:
idf=vect.idf_

In [31]:
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['hasil'])
print(dd['telapak'])  # police is most common and forecast is least common among the news headlines.

hasil telapak
1.3701673260271405
6.523458920524919


In [32]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [33]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 0.25314743 -0.07926497 -0.15779302 ...  0.07078405 -0.15720191
   0.08248597]
 [ 0.08594745 -0.03921934 -0.01889975 ... -0.0277596  -0.07017209
  -0.03474838]
 [ 0.10875014 -0.05476071 -0.01671957 ...  0.27305754 -0.01860027
  -0.00717952]
 ...
 [ 0.21358561 -0.01412773 -0.09962852 ... -0.23532446  0.11006678
   0.23077264]
 [ 0.28543252 -0.044966   -0.11713009 ... -0.01439869  0.01721985
  -0.06424741]
 [ 0.19844412 -0.07808743  0.01047994 ...  0.01968828  0.01092608
  -0.10894619]]
(500, 10)


In [34]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
    print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  25.314742581593652
Topic  1  :  -7.92649675944443
Topic  2  :  -15.779302240752369
Topic  3  :  -14.716978564412589
Topic  4  :  3.1661880386549996
Topic  5  :  -10.053559857839996
Topic  6  :  5.665605564559997
Topic  7  :  7.078404802169859
Topic  8  :  -15.720191082463902
Topic  9  :  8.248596705601583


In [35]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1000)
[[ 0.01990259  0.00868318  0.01116126 ...  0.0112083   0.00893897
   0.01253246]
 [-0.01279456  0.00508693  0.00048887 ... -0.00664785 -0.0040516
  -0.00555939]
 [-0.00221451 -0.00221382 -0.00424869 ... -0.00809025 -0.00825992
  -0.01063404]
 ...
 [-0.00339135  0.00548564  0.00708094 ...  0.01098567 -0.01600809
  -0.02055816]
 [ 0.00294209  0.0022143   0.00645995 ...  0.01137881  0.0334048
   0.05155663]
 [-0.0035613  -0.00520553 -0.00259026 ...  0.00077315 -0.03029028
  -0.04712787]]


In [36]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
citra sistem data metode game siswa nilai hasil informasi proses 

Topic 1: 
citra segmentasi fitur tahap wajah darah penyakit ekstraksi pembuluh akurasi 

Topic 2: 
game citra edukasi pembelajaran anak android permainan menarik matematika bahasa 

Topic 3: 
siswa keputusan kriteria pendukung beasiswa segmentasi game citra metode saw 

Topic 4: 
siswa pembelajaran elearning guru belajar sekolah media aplikasi materi bahasa 

Topic 5: 
wajah pengenalan siswa distance ekspresi analysis euclidean tangan pembelajaran training 

Topic 6: 
arsitektur perusahaan erp sistem zf driven penilaian enterprise model wajah 

Topic 7: 
beasiswa dokumen pencarian web bahasa informasi keputusan madura semantik sistem 

Topic 8: 
madura bahasa batik pembelajaran elearning belajar model indonesia perusahaan mahasiswa 

Topic 9: 
batik citra siswa warna pencarian sekolah tekstur isi game akademik 

